# Exercises

Create a model of a simple heat pump using R290 as refrigerant and with the
boundary conditions denoted in table below. Do not consider hot side of the
evaporator or cold side of the condenser in your model.

```{figure} /figures/heat_pump.svg
---
name: simple heat pump flowsheet
---
Flow sheet of the simple heat pump.
```

## Setup

| Location | Parameter | Value | Unit |
|:--------- |:-------- | -----:| ---- |
| 2 | Temperature | 10 | °C |
| 4 | Temperature | 60 | °C |
| Condenser | Heat transfer | 1 | MW |
| Compressor | efficiency | 80 | % |

## Tasks part 1

Calculate:

- pressure levels for evaporation and condensation. 
- COP and compressor power input.
- the Carnot factor.
- the mass flow of the refrigerant.

Create three figures:

- A logph-diagram of the cycle.
- Two diagrams indicating the dependency of the COP:
  - as function of the heat temperature level
  - as function of the heat sink temperature level

Modify your model and calculate the following:

- How would the COP of the heat pump change if
  - the fluid leaving the evaporated is superheated by 5 °C?
  - the fluid leaving the condenser is subcooled by 5 °C?
- What is the maximum heat supply temperature, that can be reached if (e.g. due
  to regulatory constraints) the cop of the heat pump has to be 6 at minimum?

In [ ]:
from CoolProp.CoolProp import PropsSI


def run_forward(fluid, t_2, t_4, eta_s, heat):

    p_2 = PropsSI("P", "T", t_2, "Q", 1, fluid)
    p_4 = PropsSI("P", "T", t_4, "Q", 0, fluid)

    h_2 = PropsSI("H", "T", t_2, "Q", 1, fluid)
    s_2 = PropsSI("S", "T", t_2, "Q", 1, fluid)

    p_3 = p_4
    h_3s = PropsSI("H", "S", s_2, "P", p_3, fluid)
    h_3 = h_2 + (h_3s - h_2) / eta_s

    h_4 = PropsSI("H", "T", t_4, "Q", 0, fluid)
    h_1 = h_4

    m = heat / (h_4 - h_3)
    power = m * (h_3 - h_2)
    return m, power, p_4, p_2


fluid = "R290"
t_2 = 283.15
t_4 = 333.15
eta_s = 0.8
heat = -1e6
m, power, p_4, p_2 = run_forward(fluid, t_2, t_4, eta_s, heat)
cop = abs(heat) / power
heat_evap = abs(heat) - power

In [ ]:
round(cop, 2)

In [ ]:
round(heat_evap)

In [ ]:
round(power)

In [ ]:
round(m, 2)

In [ ]:
round(p_2)

In [ ]:
round(p_4)

Change the working fluid in your setup to ammonia.

- How does the COP compare to the first setup? Why is it different?


In [ ]:
fluid = "NH3"
m, power, p_4, p_2 = run_forward(fluid, t_2, t_4, eta_s, heat)
cop = abs(heat) / power
heat_evap = abs(heat) - power
heat_evap

In [ ]:
round(cop, 2)

Carnot-efficiency is higher per se. Thermodynamic temperature of heat supply is lower than with the first fluid.

## Tasks part 2

The available compressor power is now limited to 200 kW.

- What amount of heat can the heat pump deliver in the conditions given in the
  table?
- What amount of heat can be delivered when the heat source temperature is
  reduced to 0 °C.
- At which heat source temperature level does the heat pump deliver 1 MW of heat
  again?


In [ ]:
power_new = 0.2e6
fluid = "R290"
power = 0
heat_guess = -1e6
d = 1e-6
while abs(power - power_new) > 1e-6:
    m, power, p_2, p_4 = run_forward(fluid, t_2, t_4, eta_s, heat_guess)
    residual = power - power_new
    heat_upper = heat_guess + d
    heat_lower = heat_guess - d
    _, power_upper, _, _ = run_forward(fluid, t_2, t_4, eta_s, heat_upper)
    _, power_lower, _, _ = run_forward(fluid, t_2, t_4, eta_s, heat_lower)
    derivative = (power_upper - power_lower) / (2 * d)
    heat_guess -= residual / derivative

heat_guess

In [ ]:
power_new = 0.2e6
fluid = "R290"
power = 0
heat = -1e6
d = 1e-6
t_4_guess = 333.15
while abs(power - power_new) > 1e-6:
    m, power, p_2, p_4 = run_forward(fluid, t_2, t_4_guess, eta_s, heat)
    residual = power - power_new
    t_4_upper = t_4_guess + d
    t_4_lower = t_4_guess - d
    _, power_upper, _, _ = run_forward(fluid, t_2, t_4_upper, eta_s, heat)
    _, power_lower, _, _ = run_forward(fluid, t_2, t_4_lower, eta_s, heat)
    derivative = (power_upper - power_lower) / (2 * d)
    t_4_guess -= residual / derivative

t_4_guess